# Intro

This notebook is a step by step instruction for completing capstone project

Part 1 goes through the work environment setup and is mostly mandatory to follow through.

Parts 2, 3 and 4 are an optional guides of how to complete the code.

Part 5 goes over final steps for the project.

After all tests are passed please create a Pull Request to main.

> **Important:** Most of the places where you need to insert code are marked with `#TODO`

# **Part 1: Setting Up the Development Environment**

Before we start writing code, we need to properly set up our development environment. This ensures that the project will run identically on any machine. We will use `pyenv` to manage Python versions and `Poetry` to manage project dependencies.

> **Note:** All the following commands should be executed in your terminal (e.g., Terminal, iTerm, or the built-in VS Code terminal), not in Jupyter Notebook cells.

---

### **Step 1: Install `pyenv` and the `pyenv-virtualenv` Plugin**

`pyenv` is a tool that allows you to easily install and switch between multiple versions of Python.

**1.1. Install `pyenv`:**
The easiest way to install `pyenv` on Linux or macOS is by running the following command in your terminal:
```bash
curl -fsSL https://pyenv.run | bash
```

**1.2. Configure Your Shell for `pyenv`:**
After installation, you need to add a few lines to your shell's configuration file (.bashrc, .zshrc, .profile, etc.) so that pyenv loads automatically. The installer usually tells you exactly which lines to add. Most often, they look like this:

```bash
echo 'export PYENV_ROOT="$HOME/.pyenv"' >> ~/.bashrc
echo '[[ -d $PYENV_ROOT/bin ]] && export PATH="$PYENV_ROOT/bin:$PATH"' >> ~/.bashrc
echo 'eval "$(pyenv init - bash)"' >> ~/.bashrc
```
**[Link to official pyenv github for installation](https://github.com/pyenv/pyenv)**

>**Important**: After this step, you must restart your terminal for the changes to take effect.


**1.3. Install the `pyenv-virtualenv` Plugin**:
This plugin allows pyenv to create and manage virtual environments, isolating our project dependencies.
```bash
git clone https://github.com/pyenv/pyenv-virtualenv.git $(pyenv root)/plugins/pyenv-virtualenv
```
**[Link to official pyenv-virtualenv github for installation](https://github.com/pyenv/pyenv-virtualenv)**

---

### **Step 2: Installation Poetry**

[Poetry docs](https://python-poetry.org/docs/#installation)

**2.1 Installation**:

The script can be executed directly (i.e. ‘curl python’) or downloaded and then executed from disk (e.g. in a CI environment)
```bash
# Linux, macOS, Windows (WSL)
curl -sSL https://install.python-poetry.org | python3 -

# Windows (Powershell)
(Invoke-WebRequest -Uri https://install.python-poetry.org -UseBasicParsing).Content | py -
```
**2.2 Add Poetry to your PATH**

The installer creates a poetry wrapper in a well-known, platform-specific directory:

```bash
echo 'export PATH="$HOME/.local/bin:$PATH"' >> ~/.bashrc
```

---

### **Step 3: Install Python & Create a Virtual Environment**

Now that `pyenv` is ready, we can install a specific version of Python and create an isolated environment for our project.

**3.1. Install the Required Python Version:**

```bash
pyenv install 3.12.3 --skip-existing
pyenv virtualenv 3.12.3 ds-capstone-venv
pyenv activate ds-capstone-venv
```

* 3.12.3 is the Python version we are using for the project.
* --skip-existing is a flag that will skip the installation if this version already exists.

> **Note:** If you have any issues with installing python, try installing all required linux packages, for example:

```bash
sudo apt install -y make build-essential libssl-dev zlib1g-dev libbz2-dev libreadline-dev libsqlite3-dev wget curl llvm libncursesw5-dev xz-utils tk-dev libffi-dev liblzma-dev python-openssl
```

**3.3. Link the Environment to Your Project:**
This is the most convenient feature of `pyenv`. We can specify that our project folder should automatically activate the correct environment.

```bash
# Navigate to your project folder
cd /path/to/your/project

# Set the local environment for this folder
pyenv local ds-capstone-venv
```

This command creates a hidden file named .python-version in the directory. Now, every time you cd into this folder in your terminal, pyenv will automatically activate the ds-capstone-venv environment.

---

### **Step 4: Set Up Project Dependencies with Poetry**
`Poetry` is a dependency manager that helps us manage the libraries our project uses.

**4.1. Configure Poetry to Use the pyenv Environment:**
his is a key step. We need to tell Poetry: "Do not create your own virtual environment; use the one that is already active (`ds-capstone-venv`)".

```bash
# Run this command inside your project folder
poetry config virtualenvs.create false --local
```

**4.2. Install Dependencies:**

Now, you can install all the project dependencies.

**The Standard Way:**
Run the following command. `Poetry` will read the `poetry.lock` file (if it exists) to install the exact versions of all libraries, ensuring a reproducible environment.

```bash
poetry install
```

**If You Encounter Conflicts or Errors...**
Sometimes, the `poetry.lock` file can become outdated or have conflicts (e.g., if you changed the dependencies in `pyproject.toml`). In this case, the best solution is to regenerate it.

```bash
rm poetry.lock
poetry install --with dev
```

---

### **Step 5: Set Up the Local LLM Server (Ollama)**

[Download Ollama](https://ollama.com/download)

Our project uses a Large Language Model running locally. We use Ollama to manage this.

**5.1. Install Ollama:**
```bash
curl -fsSL [https://ollama.com/install.sh](https://ollama.com/install.sh) | sh
```

**5.2. Download the Model:**
After installing Ollama, you need to download the specific model we'll be using.
```bash
ollama pull qwen3
```


---

# **Part 2: Semantic Search - Embeddings and FAISS**

This module is the heart of our intelligent search engine. But before we dive in, let's understand the problem we're solving.

### The Problem with Traditional Keyword Search

Imagine a standard search function (like Ctrl+F). It's great at finding exact word matches. However, it completely fails to understand the *meaning* or *context*. For example:
- A user searches for **"summer clothes"**.
- A product is described as **"a light sundress for warm weather"**.

A traditional keyword search will **fail** to find this product because the exact words don't match. This is a huge limitation for creating a user-friendly experience.

### The Solution: Searching by Meaning

**Semantic Search** overcomes this limitation by understanding the *intent* behind the words. It knows that "summer clothes" and "a light sundress" are conceptually related. To achieve this, we need a way to represent the meaning of text numerically.

This is where our two key technologies come into play:

1.  **Embeddings:** An embedding is a vector (a list of numbers) that serves as a text's "coordinate" in a high-dimensional "meaning space". A specialized AI model reads a piece of text and converts it into a vector. Texts with similar meanings will have vectors that are very close to each other in this space.

2.  **FAISS (Facebook AI Similarity Search):** Our search task now becomes a mathematical problem: given the vector for a user's query, find the product vectors that are closest to it. Comparing the query vector to every single product vector one-by-one would be incredibly slow with thousands of items. **FAISS** is a highly optimized library that builds a special index of all our product vectors, allowing it to find the "nearest neighbors" almost instantly.

### Our Plan for This Module:

Our workflow is divided into two stages:

- **Indexing (an offline step):** We will take all our product descriptions, use an embedding model to convert them into vectors, and build a FAISS index to store them efficiently.
- **Searching (an online step):** We will take a user's live query, convert it into a vector, and use our pre-built FAISS index to rapidly find the most similar products.

---

## Data for semantic search

Our semantic search engine needs data to search through. For this lab, we will use `products.csv`, a sample dataset containing information about various products.

The first step in any data science project is to load and inspect our data.

In [1]:
import pandas as pd
products = pd.read_csv('./data/products.csv')
products

,id,title,description,category,brand,color,material,price,keywords
0,1,iPhone 14 Pro,Premium smartphone running iOS 16 with 48MP ca...,phone,Apple,gray,glass/metal,1200,"iphone, ios, smartphone, apple, camera, oled"
1,2,Samsung Galaxy S23,Android 13 flagship with powerful Snapdragon p...,phone,Samsung,black,glass/metal,1100,"samsung, android, galaxy, camera, flagship"
2,3,Xiaomi Redmi Note 12,"Budget Android phone with MIUI 14, long batter...",phone,Xiaomi,blue,plastic,250,"xiaomi, android, miui, budget, display"
3,4,Google Pixel 8,Android 14 smartphone with top-tier camera and...,phone,Google,green,glass/metal,900,"pixel, android, camera, google, ai"
4,5,OnePlus 11,"Android 13 phone with fast performance, Oxygen...",phone,OnePlus,gray,glass/metal,800,"oneplus, android, oxygenos, fast, amoled"
5,6,Apple Watch Series 9,Smartwatch running watchOS 10 with fitness tra...,watch,Apple,black,aluminum,500,"apple, watchos, ios, fitness, health"
6,7,Samsung Galaxy Watch 6,"Stylish smartwatch with Android Wear OS, advan...",watch,Samsung,silver,stainless,450,"samsung, wearos, android, watch, health"
7,8,Garmin Forerunner 265,"Sports watch with Garmin OS, VO2 max tracking,...",watch,Garmin,red,plastic,400,"garmin, gps, garminos, fitness, running"
8,9,Huawei Watch GT 4,"Long-lasting smartwatch using Huawei LiteOS, w...",watch,Huawei,green,stainless,300,"huawei, liteos, wellness, battery, tracker"
9,10,Fitbit Versa 4,"Lightweight watch with Fitbit OS, focused on s...",watch,Fitbit,pink,plastic,200,"fitbit, fitbitos, health, sleep, activity"


## Preproccessing

In [2]:
import pandas as pd

def load_and_preprocess_data(file_path: str) -> pd.DataFrame:
    """
    Load and preprocess product data from a CSV file.

    This function reads a CSV file containing product information, 
    cleans the column names by removing spaces, sets the 'id' column 
    as the index, and creates a new column 'text' that combines 
    the 'title', 'description', 'category', 'brand', and 'keywords' 
    into a single string for each product.

    Parameters:
    file_path (str): The path to the CSV file containing product data.

    Returns:
    pd.DataFrame: A DataFrame containing the preprocessed product data, 
                  or an empty DataFrame if the file is not found.
    """
    try:
        products = pd.read_csv(file_path)

        products.columns = products.columns.str.replace(' ', '')
        products.set_index('id', inplace=True)

        products["text"] = products.apply(
            lambda row: f"{row['title']}. {row['description']}. {row['category']}. {row['brand']}. {row['keywords']}",
            axis=1
        )
        return products

    except FileNotFoundError:
        return pd.DataFrame()

In [3]:
products = load_and_preprocess_data('./data/products.csv')
products

,title,description,category,brand,color,material,price,keywords,text
id,,,,,,,,,
1,iPhone 14 Pro,Premium smartphone running iOS 16 with 48MP ca...,phone,Apple,gray,glass/metal,1200,"iphone, ios, smartphone, apple, camera, oled",iPhone 14 Pro . Premium smartphone ru...
2,Samsung Galaxy S23,Android 13 flagship with powerful Snapdragon p...,phone,Samsung,black,glass/metal,1100,"samsung, android, galaxy, camera, flagship",Samsung Galaxy S23 . Android 13 flagship w...
3,Xiaomi Redmi Note 12,"Budget Android phone with MIUI 14, long batter...",phone,Xiaomi,blue,plastic,250,"xiaomi, android, miui, budget, display",Xiaomi Redmi Note 12 . Budget Android phone ...
4,Google Pixel 8,Android 14 smartphone with top-tier camera and...,phone,Google,green,glass/metal,900,"pixel, android, camera, google, ai",Google Pixel 8 . Android 14 smartphone...
5,OnePlus 11,"Android 13 phone with fast performance, Oxygen...",phone,OnePlus,gray,glass/metal,800,"oneplus, android, oxygenos, fast, amoled",OnePlus 11 . Android 13 phone with...
6,Apple Watch Series 9,Smartwatch running watchOS 10 with fitness tra...,watch,Apple,black,aluminum,500,"apple, watchos, ios, fitness, health",Apple Watch Series 9 . Smartwatch running wa...
7,Samsung Galaxy Watch 6,"Stylish smartwatch with Android Wear OS, advan...",watch,Samsung,silver,stainless,450,"samsung, wearos, android, watch, health",Samsung Galaxy Watch 6 . Stylish smartwatch wi...
8,Garmin Forerunner 265,"Sports watch with Garmin OS, VO2 max tracking,...",watch,Garmin,red,plastic,400,"garmin, gps, garminos, fitness, running",Garmin Forerunner 265 . Sports watch with Gar...
9,Huawei Watch GT 4,"Long-lasting smartwatch using Huawei LiteOS, w...",watch,Huawei,green,stainless,300,"huawei, liteos, wellness, battery, tracker",Huawei Watch GT 4 . Long-lasting smartwat...


### Text Vectorization (Embeddings)
For a computer to "understand" text, we use a Transformer model to convert any text into a fixed-length vector (e.g., an array of 768 numbers). Texts with similar meanings will have vectors that are close to each other in this multi-dimensional space.

**[Documentation for vectorizing](https://huggingface.co/intfloat/multilingual-e5-large-instruct)**

Use it to fill in [vectorizer.py](./src/ds_capstone/vectorizer.py)

Vectorizer uses `intfloat/multilingual-e5-large-instruct` model, as it is small and good enough for an example, Huggingface automatically downloads it for us.

> **Important:** Run vectorizer tests to validate that code is correct:

`poetry run pytest tests/test_text_vectorizer.py`

### Building the FAISS Index
Now that we have vectors for all our products, we need a tool for fast searching. Iterating through millions of vectors manually is too slow. This is where FAISS (Facebook AI Similarity Search) comes in — a library for ultra-fast similarity search in large sets of vectors.

This step needs to be run once to prepare the data for our API.

**[Documentation for faiss](https://github.com/facebookresearch/faiss/wiki/Getting-started)**

Use it to fill in [test_search_index.py](./src/ds_capstone/search_index.py)

> **Important:** Run search index tests to validate that code is correct:

`poetry run pytest tests/test_search_index.py`

---

# **Part 3: Text Summarization with an LLM**

Having built our retrieval system (semantic search), let's now focus on a purely **generative** task: summarization. The goal is to take a potentially long piece of text and have an AI generate a concise, easy-to-read summary that captures the main ideas.

### The Power of Large Language Models (LLMs)

For this task, we will use a **Large Language Model (LLM)**. An LLM is a massive neural network trained on an incredible amount of text from the internet. This training allows it to understand grammar, context, facts, and even nuanced writing styles. Because of this deep understanding, LLMs are exceptionally good at tasks like summarization. They can read a document, identify the key points, and then generate new, shorter text in fluent, human-like language.

To run these powerful models on our own machines, we use **Ollama**. It's a fantastic tool that downloads and serves LLMs locally, giving us a private and powerful AI assistant without sending our data to the cloud.

### The Art of Asking: Prompt Engineering

The architecture for this service is straightforward:
`[Frontend (Streamlit)] <--> [Backend (FastAPI)] <--> [LLM (Ollama)]`

The most important part of this interaction happens in the backend, where we craft the request to the LLM. This is known as **prompt engineering**. The quality of the LLM's output is highly dependent on how well we formulate our prompt.

A good prompt typically consists of:
1.  **A System Message:** An instruction that sets the context and defines the AI's role (e.g., "You are an expert summarizer."). This guides the model's tone and focus.
2.  **A Human Message:** The user's input, in this case, the text that needs to be summarized.

In the next section, we will examine our `TextSummarizer` class, which uses the `LangChain` library to structure these prompts and communicate with our local Ollama model to get the job done.

### Step 1: Defining the Agent's State and Tools

Every agent built with `LangGraph` operates on a **State**. Think of the state as the agent's short-term memory or "scratchpad" for a single task. It's an object that is passed from step to step (or "node" to "node") and gets updated along the way.

For our agent, the state is simple: it just contains a list of messages that represents the "scratchpad" for the current task. The `Annotated[list, add_messages]` syntax is a special `LangGraph` helper that ensures new messages are always *added* to the list, not replaced. This is how the scratchpad gets filled.

**[Documentation for state](https://langchain-ai.github.io/langgraph/tutorials/get-started/1-build-basic-chatbot/#1-install-packages)**

**[Documentation for tools](https://langchain-ai.github.io/langgraph/tutorials/get-started/2-add-tools/#3-define-the-tool)**

### Step 2: The Agent's Logic - A Cyclic Workflow

A simple chain just goes from start to finish. An **agent** is more powerful because it can decide its next step and even loop, which mimics a reasoning process. Our agent's intelligence comes from this simple, cyclic graph.

The process works like this:

1.  **Agent Node:** The agent first receives the user's request and thinks about what to do. It might decide to answer directly (summarize), or it might decide to use a tool (get the date).
2.  **Decision (Conditional Edge):** The graph then checks the agent's decision.
    - If the agent provided a final answer, the process ends.
    - If the agent decided to use a tool, the process moves to the `tools` node.
3.  **Tools Node:** The requested tool (e.g., `get_current_date`) is executed, and its output is recorded on the scratchpad (`messages`).
4.  **Loopback:** The flow **must** return to the `agent` node. This is the most critical step! The agent now sees the original request *and* the result from the tool, allowing it to formulate a final, helpful response to the user (e.g., "Today's date is...").

This creates the following flow:

![Agent flow](./agent.png)

`START` -> `agent` -> (Does the response contain a tool call?)
- **NO:** -> `END`
- **YES:** -> `tools` -> `agent` (Loop back to think again with new info)

You have an example prompts already set up in [config.py](./config.py), I managed to get it to work with Qwen3, but it is relatively big model, so you can replace it with Gemini (if you use free $300 for Google cloud)

### Step 3: Building the Graph in Code (`_build_graph`)

This is where we define the agent's "nervous system."

**[Documentation for graph](https://langchain-ai.github.io/langgraph/tutorials/get-started/1-build-basic-chatbot/#3-add-a-node)**

### Step 4: Test agent

In [ ]:
# Import the SummarizerAgent class
import sys
import os

# Add the src directory to the Python path
sys.path.append(os.path.join(os.getcwd(), 'src'))

from ds_capstone.summarizer_graph import SummarizerAgent

# Create an instance of the SummarizerAgent
# Note: Make sure you have Ollama installed and the qwen3 model downloaded
# You can install Ollama from: https://ollama.ai/
# Then run: ollama pull qwen3

agent = SummarizerAgent(model_name="qwen3")
print("SummarizerAgent initialized successfully!")

In [ ]:
# Test the agent's tool usage capability
response = agent.execute("What is today's date?")
print("Agent Response:", response)

Expected response:

```bash
Decision: Call tools.
--- Calling get_current_date tool ---
Decision: End.
Agent Response: <think>
</think>

[summary]: The current date is Thursday, August 07, 2025.
```

> **Note:** model outputs <think> blocks, it's because this is a model with thinking, for our current project this doesn't really matter

In [ ]:
# Test text summarization
sample_text = """
Machine learning is a method of data analysis that automates analytical model building. 
It is a branch of artificial intelligence (AI) based on the idea that systems can learn 
from data, identify patterns and make decisions with minimal human intervention. Machine 
learning algorithms build a model based on sample data, known as training data, in order 
to make predictions or decisions without being explicitly programmed to do so. Machine 
learning algorithms are used in a wide variety of applications, such as in medicine, 
email filtering, speech recognition, and computer vision, where it is difficult or 
unfeasible to develop conventional algorithms to perform the needed tasks.
"""

summarization_request = f"Please summarize the following text: {sample_text}"
response = agent.execute(summarization_request)
print("Summary Response:", response)

Expected response:

```bash
Decision: End.
Summary Response: <think>
Okay, the user wants me to summarize the provided text about machine learning. Let me read through the text first.

The text explains that machine learning is a data analysis method that automates model building, part of AI. It mentions that systems learn from data, find patterns, and make decisions with little human input. It also talks about using training data to build models for predictions without explicit programming. Applications include medicine, email filtering, speech recognition, and computer vision, where traditional algorithms are hard to develop.

I need to create a concise summary. Let me identify the key points: definition, relation to AI, learning from data, model building, applications. Avoid specific examples unless necessary. Keep it brief. Let me check if the user wants any specific format. The instructions say to use the summary format [summary]: ... So I'll structure it that way. Make sure it's a short representation. Alright, putting it all together.
</think>

[summary]: Machine learning is a data analysis method in AI that automates model building by enabling systems to learn from data, identify patterns, and make decisions with minimal human intervention. It uses training data to create predictive models without explicit programming, applied in fields like medicine, email filtering, speech recognition, and computer vision.
```

> **Note:** your response may slightly differ, as LLM models are not deterministic

> **Important:** run tests for summarizer:

`poetry run pytest tests/test_text_summarizer.py`

# Part 4: API

API is straightforward, just put everything together in [api.py](./api.py)

- `lifespan` and `read_root` functions are already set up for you, no need to change anything
- `summarize_text` function should execute LLM agent
- `semantic_search` function should create embedding for the input query and return closest items from index

Use Streamlit to test it:

`poetry run honcho start`

This will start Streamlit locally - [http://localhost:8501](http://localhost:8501)

> **Important:** Run API tests to validate that code is correct:

`poetry run pytest tests/test_api.py`

# Part 5: Run tests and commit changes

## Run isort formatting
`poetry run isort .`

## Run black formatting
`poetry run black .`

## Check isort formatting
`poetry run isort . --check-only`

## Check black formatting
`poetry run black . --check`

## Check flake8
`poetry run flake8 .`

## Check mypy formatting
`poetry run mypy .`

## Run all tests
`poetry run pytest`

## If all passed feel free to commit changes and create Pull Request